# MatchPredictor

### A neural network based Premier League match outcome predictor

#### Importing Libraries & Packages

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


#### Importing Match Data

In [5]:
# Imported CSV becomes a pandas dataframe object
data_19_20 = pd.read_csv("Prem_data_19-20\england-premier-league-matches-2019-to-2020-stats.csv")
data_18_19 = pd.read_csv("Prem_data_18-19\england-premier-league-matches-2018-to-2019-stats.csv")
data_17_18 = pd.read_csv("Prem_data_17-18\england-premier-league-matches-2017-to-2018-stats.csv")
data_16_17 = pd.read_csv("Prem_data_16-17\england-premier-league-matches-2016-to-2017-stats.csv")
data_15_16 = pd.read_csv("Prem_data_15-16\england-premier-league-matches-2015-to-2016-stats.csv")

#### Preprocessing Data

In [6]:
col_names = data_19_20.columns
num_rows = data_19_20.shape[0]

print(col_names)
print(num_rows)

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

#### Multi-Layer Perceptron Model

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class MultiLayerPerceptron(nn.Module):

    def __init__(self, input_size):

        super(MultiLayerPerceptron, self).__init__()

        self.fc1 = nn.Linear(input_size,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,1)

    def forward(self, features):

        features = F.relu(self.fc1(features))
        features = F.relu(self.fc2(features))
        features = F.relu(self.fc3(features))
        features = F.relu(self.fc4(features))
        features = torch.sigmoid(features)

        return features